In [18]:
from IPython.display import display, Markdown  # 在jupyter显示信息的工具

from langchain.chains import RetrievalQA #检索QA链，在文档上进行检索
# from langchain.chat_models import ChatOpenAI #openai模型
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain.indexes import VectorstoreIndexCreator #导入向量存储索引创建器
from langchain.vectorstores import DocArrayInMemorySearch #向量存储
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings  # ollama嵌入

import pandas as pd

In [17]:
# 加载中文数据
file = "../data/product_data.csv"
loader = CSVLoader(file_path=file)
data = loader.load()

# 查看数据
test_data = pd.read_csv(file, skiprows=0)
display(test_data.shape)
display(test_data.head())

(25, 2)

,product_name,description
0,全自动咖啡机,规格:\n大型 - 尺寸：13.8'' x 17.3''。\n中型 - 尺寸：11.5'' ...
1,电动牙刷,规格:\n一般大小 - 高度：9.5''，宽度：1''。\n\n为什么我们热爱它:\n我们的...
2,橙味维生素C泡腾片,规格:\n每盒含有20片。\n\n为什么我们热爱它:\n我们的橙味维生素C泡腾片是快速补充维...
3,无线蓝牙耳机,规格:\n单个耳机尺寸：1.5'' x 1.3''。\n\n为什么我们热爱它:\n这款无线蓝...
4,瑜伽垫,规格:\n尺寸：24'' x 68''。\n\n为什么我们热爱它:\n我们的瑜伽垫拥有出色的...


In [13]:
# 创建 Ollama 嵌入模型实例
embeddings = OllamaEmbeddings(model="qwen2.5")  # 或其他在 Ollama 中安装的模型名称
# 将指定向量存储类,创建完成后，我们将从加载器中调用,通过文档记载器列表加载
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch, embedding=embeddings,  # 添加 embedding 参数
).from_loaders(
    [loader]
)

# 通过指定语言模型、链类型、检索器和我们要打印的详细程度来创建检索QA链
llm = ChatOllama(model="qwen2.5", temperature=0.0)  # 使用ollama模型
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs={"document_separator": "<<<<>>>>>"},
)

/opt/anaconda3/envs/llm_env/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')
